# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# gmaps were having trouble displaying the map, it was showing "This page can't load Google Maps correctly" notification.
# Adding this code to help display the maps

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv('../WeatherPy/weather_df.csv')
weather_df.dropna(inplace=True)
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-philippe,-21.3585,55.7679,84.20,65,0,24.16,RE,1616052868
1,1,nizhniy kuranakh,58.8353,125.4869,23.68,80,100,5.70,RU,1616052868
2,2,rikitea,-23.1203,-134.9692,75.24,86,100,16.08,PF,1616052869
3,3,arraial do cabo,-22.9661,-42.0278,75.49,89,70,17.85,BR,1616052870
4,4,bilibino,68.0546,166.4372,-16.40,90,0,3.02,RU,1616052389


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)

In [5]:
# Find the highest humidity for the max_intensity

max_humidity = weather_df["Humidity"].max()
max_humidity

100

In [6]:
# Plot the heatmap

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
vacation_df = weather_df.loc[(weather_df["Max Temp"] < 80) &
                          (weather_df["Max Temp"] > 70) &
                          (weather_df["Wind Speed"] < 10) &
                          (weather_df["Cloudiness"] == 0)]

vacation_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kruisfontein,-34.0033,24.7314,73.99,78,0,3.00,ZA,1616052899
55,55,hermanus,-34.4187,19.2345,71.01,75,0,4.00,ZA,1616052907
135,135,faya,18.3851,42.4509,77.00,15,0,9.22,SA,1616052966
182,182,taoudenni,22.6783,-3.9836,71.33,19,0,7.74,ML,1616053000
190,190,vila velha,-20.3297,-40.2925,79.00,94,0,2.30,BR,1616052498
357,357,puerto escondido,15.8500,-97.0667,74.82,73,0,4.90,MX,1616053128
578,578,sao filipe,14.8961,-24.4956,70.93,75,0,6.46,CV,1616053295
623,623,conakry,9.5380,-13.6773,76.93,66,0,3.83,GN,1616053351
625,625,iquitos,-3.7481,-73.2472,73.40,94,0,1.74,PE,1616053353


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = vacation_df.copy()

hotel_df["Hotel Name"] = ""

hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,45,kruisfontein,-34.0033,24.7314,73.99,78,0,3.00,ZA,1616052899,
55,55,hermanus,-34.4187,19.2345,71.01,75,0,4.00,ZA,1616052907,
135,135,faya,18.3851,42.4509,77.00,15,0,9.22,SA,1616052966,
182,182,taoudenni,22.6783,-3.9836,71.33,19,0,7.74,ML,1616053000,
190,190,vila velha,-20.3297,-40.2925,79.00,94,0,2.30,BR,1616052498,
357,357,puerto escondido,15.8500,-97.0667,74.82,73,0,4.90,MX,1616053128,
578,578,sao filipe,14.8961,-24.4956,70.93,75,0,6.46,CV,1616053295,
623,623,conakry,9.5380,-13.6773,76.93,66,0,3.83,GN,1616053351,
625,625,iquitos,-3.7481,-73.2472,73.40,94,0,1.74,PE,1616053353,


In [9]:
# Set params dictionary to update each iteration

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.


In [10]:
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,45,kruisfontein,-34.0033,24.7314,73.99,78,0,3.00,ZA,1616052899,Oyster Bay House Rental
55,55,hermanus,-34.4187,19.2345,71.01,75,0,4.00,ZA,1616052907,Misty Waves Boutique Hotel
135,135,faya,18.3851,42.4509,77.00,15,0,9.22,SA,1616052966,مقبرة ال مجمل
182,182,taoudenni,22.6783,-3.9836,71.33,19,0,7.74,ML,1616053000,
190,190,vila velha,-20.3297,-40.2925,79.00,94,0,2.30,BR,1616052498,Quality Suites Vila Velha


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

## Observation

#### From the heatmap, we could clear identify the cities near the sea are showing higher humidity. This might because citieis closer to the sea and oceans are humid as a result of their proximity to the water bodies.